<a href="https://colab.research.google.com/github/tssriram/web_scraping/blob/main/masters_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import json  
from pandas.io.json import json_normalize 
import asyncio
#!pip install aiohttp
import aiohttp
import nest_asyncio
nest_asyncio.apply()
from asyncio import ensure_future, gather
#!pip install flatten-json
from flatten_json import flatten

In [ ]:
async def request_controller(urls):
    session_timeout = aiohttp.ClientTimeout(total=None)
    async with aiohttp.ClientSession(timeout=session_timeout) as session:
        tasks = [ensure_future(request_worker(session, url)) for url in urls]
        results = await gather(*tasks)
    return results

async def request_worker(session, url):
    async with session.get(url) as response:
        pbar.update(1)
        return await response.json()

In [ ]:
urls=[]
url_a='https://search-facets.prtl.co/?q=lv-master|uc-108|tc-EUR&facets=["ci"]'
params=requests.get(url_a)

country = json_normalize(params.json()['ci']).transpose()
country.index=country.index.astype(int)

In [ ]:
sum(country.iloc[:,0])

79738

In [ ]:
for t in country.index: 
  url_b='https://search-facets.prtl.co/?q=ci-%s|lv-master|uc-108|tc-EUR&facets=["di"]' % (t)
  params=requests.get(url_b)

  category = json_normalize(params.json()['di']).transpose()
  category.index=category.index.astype(int)

  for i in category.index:
    for j in np.arange(0,int(category.loc[i]),100):
      urls.append('https://search.prtl.co/2018-07-23/?q=en-2425|ci-%s|di-%s|lv-master&size=100&start=%s' % (t,i,j))

In [ ]:
limit=len(urls)
pbar=tqdm(total=limit);

21044it [1:02:41,  5.59it/s]

In [ ]:
loop = asyncio.get_event_loop()
results = loop.run_until_complete(request_controller(urls))
tqdm._instances.clear()

100%|█████████▉| 12303/12316 [02:05<00:00, 254.29it/s]

In [ ]:
data=pd.DataFrame()
l=[]
for h in tqdm(range(0,len(results))):  

  if results[h] != []:
    df=pd.DataFrame([flatten(d) for d in results[h]])

    if 'fulltime_duration_value' not in df:
      df['fulltime_duration_value'] = np.nan 
    if 'fulltime_duration_unit' not in df:
      df['fulltime_duration_unit'] = np.nan
    if 'tuition_fee_value' not in df:
      df['tuition_fee_value'] = np.nan
    if 'tuition_fee_unit' not in df:
      df['tuition_fee_unit'] = np.nan
    if 'tuition_fee_currency' not in df:
      df['tuition_fee_currency'] = np.nan
    if 'organisation' not in df:
      df['organisation'] = np.nan
    if 'venues_0_city' not in df:
      df['venues_0_city'] = np.nan
    if 'venues_0_country' not in df:
      df['venues_0_country'] = np.nan
    if 'venues_0_area' not in df:
      df['venues_0_area'] = np.nan
    if 'venues_1_city' not in df:
      df['venues_1_city'] = np.nan
    if 'venues_1_country' not in df:
      df['venues_1_country'] = np.nan
    if 'venues_1_area' not in df:
      df['venues_1_area'] = np.nan
    if 'parttime_duration_value' not in df:
      df['parttime_duration_value'] = np.nan
    if 'parttime_duration_unit' not in df:
      df['parttime_duration_unit'] = np.nan

    df=df[['title','level','degree','organisation','tuition_fee_value','tuition_fee_unit','tuition_fee_currency','fulltime_duration_value','fulltime_duration_unit',
           'density_parttime','density_fulltime','methods_face2face','methods_online','methods_blended','venues_0_city','venues_0_country','venues_0_area','venues_1_city',
           'venues_1_country','venues_1_area','parttime_duration_value','parttime_duration_unit']]
    l.append(df)

data=pd.concat(l, ignore_index=True)

100%|██████████| 12316/12316 [01:05<00:00, 186.92it/s]


In [ ]:
data=data.drop_duplicates()

In [ ]:
len(data)

78858

In [ ]:
from google.colab import files

data=data.reset_index(drop=True) 
data.to_excel('masters.xlsx',index=False)
files.download('masters.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>